In [12]:
import pandas as pd
import numpy as np

In [153]:
data=pd.read_csv('house_pred.csv')
data.shape

(1460, 81)

In [154]:
numerical_columns = data.select_dtypes(include=['float64', 'int64']).columns
data1=data.copy()
for col in numerical_columns:
    
    Q1 = data1[col].quantile(0.25)
    Q3 = data1[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data1[col][(data1[col] < lower_bound) | (data1[col] > upper_bound)] = data1[col].median()
data1.shape


C:\Users\harii\AppData\Local\Temp\ipykernel_10744\3841190075.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1[col][(data1[col] < lower_bound) | (data1[col] > upper_bound)] = data1[col].median()


(1460, 81)

In [164]:

categorical_columns = data1.select_dtypes(include=['object']).columns
column_null_counts = data1[categorical_columns].isnull().sum()
sorted_columns = column_null_counts.sort_values(ascending=False)
data2=data1.drop(columns=['PoolQC','MiscFeature','Alley','Fence','FireplaceQu'])
categorical_columns = data2.select_dtypes(include=['object']).columns
for col in categorical_columns:
    mode_value = data2[col].mode()[0]
    data2[col].fillna(mode_value, inplace=True)

data2.shape
# data2=data1.dropna(subset=categorical_columns, inplace=False)
# data2.head()

(1460, 76)

In [67]:
# categorical_columns = data2.select_dtypes(include=['object']).columns
# data3=data2.dropna(subset=categorical_columns, inplace=False)
# data3.shape

(1460, 76)

In [165]:
for col in numerical_columns:
    mode_value = data3[col].mode()[0]
    data3[col].fillna(mode_value, inplace=True)

print(data3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   float64
 5   Street         1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [166]:
categorical_columns = data3.select_dtypes(include=['object']).columns
data4=pd.get_dummies(data3,columns=categorical_columns,drop_first=True)
dependant_col='SalePrice'
cols=[col for col in data4.columns if col != dependant_col]
cols=cols+[dependant_col]
data4=data4[cols]
data4.shape

(1460, 234)

In [167]:
x_train = data4.iloc[:,:-1].values
y_train = data4.iloc[:,-1].values
x_train=pd.DataFrame(x_train)
dependant_col='SalePrice'
cols=[col for col in data4.columns if col != 'SalePrice']
x_train.columns=cols
x_train.shape

(1460, 233)

In [168]:
correlation_matrix = x_train.corr()
highly_correlated_vars = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.6:
            colname = correlation_matrix.columns[i]
            highly_correlated_vars.add(colname)

x_train = x_train.drop(columns=highly_correlated_vars)
x_train.shape

(1460, 186)

In [169]:
from statsmodels.stats.outliers_influence import \
variance_inflation_factor

vif = pd.DataFrame()
vif['variable']= x_train.columns

vif['vif'] = [variance_inflation_factor(x_train.values,i)
 for i in range(x_train.shape[1])]

vif

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1736: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


,variable,vif
0,Id,1.143218
1,MSSubClass,4.383049
2,LotFrontage,2.047188
3,LotArea,2.803582
4,OverallQual,4.336001
...,...,...
181,SaleType_New,1.608956
182,SaleType_Oth,1.074175
183,SaleCondition_AdjLand,1.369542
184,SaleCondition_Alloca,1.373609


In [180]:
data_train=pd.read_csv("test.csv")
data_price=pd.read_csv("house_pred.csv")

SalePrice=data_price['SalePrice'].iloc[-256:].tolist()
data_train['SalePrice']=SalePrice
data_train.shape

(256, 81)

In [181]:
numerical_columns = data_train.select_dtypes(include=['float64', 'int64']).columns
data_train1=data_train.copy()
for col in numerical_columns:
    Q1 = data_train1[col].quantile(0.25)
    Q3 = data_train1[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data_train1[col][(data_train1[col] < lower_bound) | (data_train1[col] > upper_bound)] = data1[col].mode()[0]

    
data_train1.shape

C:\Users\harii\AppData\Local\Temp\ipykernel_10744\3804933551.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train1[col][(data_train1[col] < lower_bound) | (data_train1[col] > upper_bound)] = data1[col].mode()[0]
C:\Users\harii\AppData\Local\Temp\ipykernel_10744\3804933551.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train1[col][(data_train1[col] < lower_bound) | (data_train1[col] > upper_bound)] = data1[col].mode()[0]
C:\Users\harii\AppData\Local\Temp\ipykernel_10744\3804933551.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in t

(256, 81)

In [182]:
data_train2=data_train1.drop(columns=['PoolQC','MiscFeature','Alley','Fence','FireplaceQu'])
for col in categorical_columns:
    mode_value = data2[col].mode()[0]
    data_train2[col].fillna(mode_value, inplace=True)

print(data_train2[categorical_columns].isna().sum())

MSZoning         0
Street           0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinType2     0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
KitchenQual      0
Functional       0
GarageType       0
GarageFinish     0
GarageQual       0
GarageCond       0
PavedDrive       0
SaleType         0
SaleCondition    0
dtype: int64


In [185]:
data_train3=data_train2.copy()
for col in numerical_columns:
    mode_value = data_train3[col].mode()[0]
    data_train3[col].fillna(mode_value, inplace=True)

print(data_train3[numerical_columns].isna().sum())

Id               0
MSSubClass       0
LotFrontage      0
LotArea          0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
MasVnrArea       0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageYrBlt      0
GarageCars       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
SalePrice        0
dtype: int64


In [186]:
# #drop rows with null values
# categorical_columns = data_train3.select_dtypes(include=['object']).columns
# data_train3=data_train3.dropna(subset=categorical_columns, inplace=False)
# data_train3.shape

In [187]:
data_train3.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             256 non-null    int64  
 1   MSSubClass     256 non-null    int64  
 2   MSZoning       256 non-null    object 
 3   LotFrontage    256 non-null    float64
 4   LotArea        256 non-null    float64
 5   Street         256 non-null    object 
 6   LotShape       256 non-null    object 
 7   LandContour    256 non-null    object 
 8   Utilities      256 non-null    object 
 9   LotConfig      256 non-null    object 
 10  LandSlope      256 non-null    object 
 11  Neighborhood   256 non-null    object 
 12  Condition1     256 non-null    object 
 13  Condition2     256 non-null    object 
 14  BldgType       256 non-null    object 
 15  HouseStyle     256 non-null    object 
 16  OverallQual    256 non-null    int64  
 17  OverallCond    256 non-null    int64  
 18  YearBuilt 

In [188]:
categorical_columns = data_train3.select_dtypes(include=['object']).columns
data_train4=pd.get_dummies(data_train3,columns=categorical_columns,drop_first=True)
dependant_col='SalePrice'
cols=[col for col in data_train4.columns if col != dependant_col]
cols=cols+[dependant_col]
data_train4=data_train4[cols]
data_train4.shape

(256, 195)

In [189]:
x_test = data_train4.iloc[:,:-1].values
y_test = data_train4.iloc[:,-1].values
x_test=pd.DataFrame(x_test)
dependant_col='SalePrice'
cols=[col for col in data_train4.columns if col != 'SalePrice']
x_test.columns=cols
x_test.shape

(256, 194)

In [191]:
from statsmodels.stats.outliers_influence import \
variance_inflation_factor
vif = pd.DataFrame()
vif['variable']= x_test.columns

vif['vif'] = [variance_inflation_factor(x_test.values,i)
 for i in range(x_test.shape[1])]

vif

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1736: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


,variable,vif
0,Id,2.170289
1,MSSubClass,6.409208
2,LotFrontage,3.202354
3,LotArea,6.675718
4,OverallQual,8.397072
...,...,...
142,PavedDrive_P,3.727187
143,PavedDrive_Y,3.917157
144,SaleType_CWD,4.066604
145,SaleType_New,2.352985


In [190]:
correlation_matrix = x_test.corr()
highly_correlated_vars = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.6:
            colname = correlation_matrix.columns[i]
            highly_correlated_vars.add(colname)


x_test = x_test.drop(columns=highly_correlated_vars)
x_test.shape

(256, 147)

In [192]:
common_columns = x_train.columns.intersection(x_test.columns)
print(x_train.shape,x_test.shape)
x_train = x_train[common_columns]
x_test = x_test[common_columns]
print(x_train.shape,x_test.shape)

(1460, 186) (256, 147)
(1460, 142) (256, 142)


In [193]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

reg.fit(x_train,y_train)

LinearRegression()

In [194]:

y_pred = reg.predict(x_test)

In [198]:
from sklearn.metrics import r2_score, mean_squared_error
r2_score(y_test,y_pred)


0.8006740128921359

In [199]:
import math
math.sqrt(mean_squared_error(y_test,y_pred))


26233.773077709975